## Graphsage 采样代码实践

GraphSage的PGL完整代码实现位于 [PGL/examples/graphsage/](https://github.com/PaddlePaddle/PGL/tree/main/examples/graphsage)

本次实践将带领大家尝试实现一个简单的graphsage 采样代码实现。

In [1]:
# 安装依赖
# !pip install paddlepaddle==1.8.4
!pip install pgl -q

## 1. 构建graph

在实现graphsage采样之前，我们需要构建一个图网络。

图网络的构建需要用到Graph类，Graph类的具体实现可以参考 [PGL/pgl/graph.py](https://github.com/PaddlePaddle/PGL/blob/main/pgl/graph.py)

下面我们简单展示一下如何构建一个图网络：

In [2]:
import random
import numpy as np
import pgl
import display

In [3]:
def build_graph():
    # 定义节点的个数；每个节点用一个数字表示，即从0~9
    num_node = 16
    # 添加节点之间的边，每条边用一个tuple表示为: (src, dst)
    edge_list = [(2, 0), (1, 0), (3, 0),(4, 0), (5, 0), 
             (6, 1), (7, 1), (8, 2), (9, 2), (8, 7),
             (10, 3), (4, 3), (11, 10), (11, 4), (12, 4),
             (13, 5), (14, 5), (15, 5)]

    g = pgl.graph.Graph(num_nodes = num_node, edges = edge_list)

    return g

# 创建一个图对象，用于保存图网络的各种数据。
g = build_graph()
display.display_graph(g)

## 2. GraphSage采样函数实现

GraphSage的作者提出了采样算法来使得模型能够以Mini-batch的方式进行训练，算法伪代码见[论文](https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf)附录A。

- 假设我们要利用中心节点的k阶邻居信息，则在聚合的时候，需要从第k阶邻居传递信息到k-1阶邻居，并依次传递到中心节点。
- 采样的过程刚好与此相反，在构造第t轮训练的Mini-batch时，我们从中心节点出发，在前序节点集合中采样$N_t$个邻居节点加入采样集合。
- 接着将邻居节点作为新的中心节点继续进行第t-1轮训练的节点采样，以此类推。
- 最后将采样到的节点和边一起构造得到子图。


In [4]:
def traverse(item):
    """traverse
    """
    if isinstance(item, list) or isinstance(item, np.ndarray):
        for i in iter(item):
            for j in traverse(i):
                yield j
    else:
        yield item

def flat_node_and_edge(nodes):
    """这个函数的目的是为了将 list of numpy array 扁平化成一个list
    例如： [array([7, 8, 9]), array([11, 12]), array([13, 15])] --> [7, 8, 9, 11, 12, 13, 15]
    """
    nodes = list(set(traverse(nodes)))
    return nodes

def graphsage_sample(graph, start_nodes, sample_num):
    subgraph_edges = []
    # pre_nodes: a list of numpy array, 
    pre_nodes = graph.sample_predecessor(start_nodes, sample_num)

    # 根据采样的子节点， 恢复边
    for dst_node, src_nodes in zip(start_nodes, pre_nodes):
        for node in src_nodes:
            subgraph_edges.append((node, dst_node))

    # flat_node_and_edge： 这个函数的目的是为了将 list of numpy array 扁平化成一个list
    # [array([7, 8, 9]), array([11, 12]), array([13, 15])] --> [7, 8, 9, 11, 12, 13, 15]
    subgraph_nodes = flat_node_and_edge(pre_nodes)

    return subgraph_nodes, subgraph_edges


In [5]:
seed = 458
np.random.seed(seed)
random.seed(seed)

start_nodes = [0]

layer1_nodes, layer1_edges = graphsage_sample(g, start_nodes, sample_num=3)
print('layer1_nodes: ', layer1_nodes)
print('layer1_edges: ', layer1_edges)
display.display_subgraph(g, {'orange': layer1_nodes}, {'orange': layer1_edges})

In [6]:
layer2_nodes, layer2_edges = graphsage_sample(g, layer1_nodes, sample_num=2)
print('layer2_nodes: ', layer2_nodes)
print('layer2_edges: ', layer2_edges)
display.display_subgraph(g, {'orange': layer1_nodes, 'Thistle': layer2_nodes}, {'orange': layer1_edges, 'Thistle': layer2_edges})